In [75]:
import tensorflow as tf
tf.__version__
tf.config.experimental.enable_op_determinism()
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
  tf.config.experimental.set_memory_growth(gpu, True)

In [76]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error
import random

In [77]:
subject = 'Sergipe - Consumo de Cimento (t)'
split_index = 191 #Referente aos 230 anos de input  
train_split = split_index + 1 -30

In [80]:
data = pd.read_csv('2003_mo_model_input_SE.csv')
data = data[[col for col in data.columns if col != subject] + [subject]] #Seta consumo (target) para a coluna final
data = data.drop([' NFSP - Fluxo Mensal (Milhões de reais)'], axis=1)
# data['Unnamed: 0'] = data['Unnamed: 0'].str[5:].astype(float)
data

,Unnamed: 0,Sergipe - IDH,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Sergipe - value,Sergipe - Produção de Cimento (t),Sergipe - PIB - Estadual,Sergipe - PIB - Construção Civil,Sergipe - PIB - Per Capita,Sergipe - PIB - Preços de Mercado,Sergipe - Desemprego,Sergipe - Consumo de Cimento (t)
0,2003-1,0.669217,0.724032,11.520143,1.639718,1.036534,1.772069e+08,7.330309e+06,0.969649,0.339447,152.791400,2.159732e+07,1.260978e+06,9.960678,2.029642e+07,8.389943,23.616
1,2003-2,0.669542,0.690297,11.189862,1.378899,0.993449,1.773884e+08,7.335910e+06,0.950783,0.341257,151.964054,2.162153e+07,1.262134e+06,9.961956,2.030238e+07,8.384067,21.357
2,2003-3,0.669868,0.669681,10.820792,1.924317,0.973020,1.775699e+08,7.341511e+06,0.938332,0.343016,154.467758,2.164574e+07,1.263289e+06,9.963234,2.030833e+07,8.378190,20.059
3,2003-4,0.670193,0.660494,10.417840,1.331174,0.940489,1.777514e+08,7.347112e+06,0.926401,0.344722,155.913400,2.166995e+07,1.264444e+06,9.964512,2.031429e+07,8.372313,19.920
4,2003-5,0.670519,0.648337,9.959690,1.736072,0.917493,1.779329e+08,7.352713e+06,0.951683,0.346174,157.467159,2.169416e+07,1.265599e+06,9.965790,2.032025e+07,8.366437,19.962
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
235,2022-8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.557749,169.752591,NaN,NaN,NaN,NaN,NaN,34.037
236,2022-9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.556659,169.621865,NaN,NaN,NaN,NaN,NaN,32.537
237,2022-10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.555237,169.855545,NaN,NaN,NaN,NaN,NaN,35.215
238,2022-11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.553445,170.188727,NaN,NaN,NaN,NaN,NaN,34.707


In [82]:
input_data = data.iloc[:split_index + 1,1:-1]
mean = np.mean(input_data, axis=0)
stddev =  np.std(input_data, axis=0)
input_data = ((input_data - mean) /stddev)
input_data

,Sergipe - IDH,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Sergipe - value,Sergipe - Produção de Cimento (t),Sergipe - PIB - Estadual,Sergipe - PIB - Construção Civil,Sergipe - PIB - Per Capita,Sergipe - PIB - Preços de Mercado,Sergipe - Desemprego
0,-2.191597,2.723741,4.398348,2.052494,3.890153,-2.042341,-2.389042,3.122582,-1.329544,-1.638248,-1.769739,-0.221258,0.566843,-2.721827,-0.858763
1,-2.153964,2.350880,4.222509,1.242280,3.551840,-2.014760,-2.352139,2.970356,-1.294452,-1.660220,-1.748579,-0.190897,0.585366,-2.626858,-0.861105
2,-2.116331,2.123016,4.026019,2.936580,3.391423,-1.987179,-2.315236,2.869895,-1.260345,-1.593729,-1.727419,-0.160535,0.603889,-2.531890,-0.863447
3,-2.078698,2.021477,3.811492,1.094024,3.135979,-1.959598,-2.278333,2.773628,-1.227259,-1.555337,-1.706259,-0.130173,0.622411,-2.436921,-0.865789
4,-2.041065,1.887113,3.567576,2.351810,2.955412,-1.932017,-2.241431,2.977624,-1.199102,-1.514074,-1.685099,-0.099812,0.640934,-2.341953,-0.868132
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
187,1.375707,-2.010387,-0.572934,-1.281958,0.589021,1.365911,0.389193,-1.749976,0.926582,-1.147300,1.088440,-1.830495,-1.273118,-0.375044,1.205390
188,1.367460,-1.870713,-0.588777,-1.358588,1.043728,1.376610,0.370392,-1.593005,0.922177,-1.137939,1.080037,-1.823605,-1.263526,-0.370593,1.201393
189,1.359213,-1.806230,-0.612606,-1.511565,1.387010,1.387308,0.351592,-1.351489,0.923529,-1.097718,1.071634,-1.816716,-1.253934,-0.366142,1.197396
190,1.350966,-1.727496,-0.640956,-1.421708,1.815728,1.398006,0.332791,-1.198492,0.936477,-1.086004,1.063231,-1.809827,-1.244342,-0.361691,1.193399


In [83]:
# Shift para prever futuro e não presente
target_data = data[subject].shift(-12)
target_data

0      19.642
1      17.778
2      21.180
3      18.824
4      16.535
        ...  
235       NaN
236       NaN
237       NaN
238       NaN
239       NaN
Name: Sergipe - Consumo de Cimento (t), Length: 240, dtype: float64

In [84]:
# input para treinamento
train_input = input_data.iloc[:train_split]
train_input

,Sergipe - IDH,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Sergipe - value,Sergipe - Produção de Cimento (t),Sergipe - PIB - Estadual,Sergipe - PIB - Construção Civil,Sergipe - PIB - Per Capita,Sergipe - PIB - Preços de Mercado,Sergipe - Desemprego
0,-2.191597,2.723741,4.398348,2.052494,3.890153,-2.042341,-2.389042,3.122582,-1.329544,-1.638248,-1.769739,-0.221258,0.566843,-2.721827,-0.858763
1,-2.153964,2.350880,4.222509,1.242280,3.551840,-2.014760,-2.352139,2.970356,-1.294452,-1.660220,-1.748579,-0.190897,0.585366,-2.626858,-0.861105
2,-2.116331,2.123016,4.026019,2.936580,3.391423,-1.987179,-2.315236,2.869895,-1.260345,-1.593729,-1.727419,-0.160535,0.603889,-2.531890,-0.863447
3,-2.078698,2.021477,3.811492,1.094024,3.135979,-1.959598,-2.278333,2.773628,-1.227259,-1.555337,-1.706259,-0.130173,0.622411,-2.436921,-0.865789
4,-2.041065,1.887113,3.567576,2.351810,2.955412,-1.932017,-2.241431,2.977624,-1.199102,-1.514074,-1.685099,-0.099812,0.640934,-2.341953,-0.868132
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
157,1.387754,-0.214006,-0.607704,0.059280,-1.233012,1.031384,0.819304,-0.883659,1.521513,-0.132792,1.143453,-1.270245,-1.549275,-0.543399,1.342411
158,1.390522,-0.434717,-0.620523,0.222389,-1.299304,1.042716,0.808136,-0.950771,1.498482,-0.189562,1.147046,-1.306762,-1.539477,-0.525912,1.335629
159,1.393291,-0.524091,-0.631530,0.191929,-1.248662,1.054049,0.796969,-1.028465,1.475235,-0.253067,1.150639,-1.343278,-1.529680,-0.508425,1.328848
160,1.396059,-0.614500,-0.640320,0.385687,-1.068274,1.065381,0.785801,-1.103668,1.452580,-0.330172,1.154232,-1.379795,-1.519882,-0.490938,1.322066


In [85]:
# Alvo para treinamento
train_target = target_data.iloc[:train_split]
train_target

0      19.642
1      17.778
2      21.180
3      18.824
4      16.535
        ...  
157    37.579
158    48.016
159    38.263
160    40.413
161    34.053
Name: Sergipe - Consumo de Cimento (t), Length: 162, dtype: float64

In [86]:
#input de test (Ano 2021)
test_input = input_data.iloc[train_split:]
test_input

,Sergipe - IDH,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Sergipe - value,Sergipe - Produção de Cimento (t),Sergipe - PIB - Estadual,Sergipe - PIB - Construção Civil,Sergipe - PIB - Per Capita,Sergipe - PIB - Preços de Mercado,Sergipe - Desemprego
162,1.401596,-0.601510,-0.653307,0.157879,-1.255326,1.088045,0.763466,-1.213929,1.414043,-0.454209,1.161418,-1.452829,-1.500286,-0.455964,1.308503
163,1.404365,-0.786068,-0.653814,0.548855,-1.118679,1.099377,0.752299,-1.292173,1.394507,-0.503123,1.165011,-1.489345,-1.490488,-0.438477,1.301721
164,1.407133,-0.830387,-0.645656,0.385685,-0.936336,1.110709,0.741131,-1.324219,1.367731,-0.562332,1.168604,-1.525862,-1.480691,-0.420990,1.294940
165,1.409902,-0.801089,-0.634274,0.420965,-0.931790,1.122042,0.729964,-1.344446,1.339914,-0.617004,1.172197,-1.562379,-1.470893,-0.403503,1.288158
166,1.412671,-0.959917,-0.624035,0.548870,-1.168522,1.133374,0.718796,-1.381638,1.313790,-0.680564,1.175790,-1.598896,-1.461095,-0.386016,1.281377
167,1.415439,-1.022309,-0.619791,0.222499,-1.285611,1.144706,0.707629,-1.411208,1.287929,-0.725299,1.179383,-1.635412,-1.451297,-0.368529,1.274595
168,1.418208,-1.074401,-0.620112,-0.614754,-1.446316,1.156038,0.696461,-1.412953,1.259641,-0.772009,1.182976,-1.671929,-1.441499,-0.351042,1.267813
169,1.419477,-1.119597,-0.615973,-0.478229,-1.357832,1.167287,0.681823,-1.491464,1.230064,-0.819918,1.179999,-1.689162,-1.433063,-0.355638,1.264943
170,1.420746,-1.078648,-0.609777,-0.393157,-1.235682,1.178536,0.667184,-1.573805,1.206835,-0.850964,1.177023,-1.706394,-1.424626,-0.360235,1.262073
171,1.422015,-1.055426,-0.602141,-0.557552,-1.065340,1.189784,0.652545,-1.564950,1.184830,-0.892669,1.174047,-1.723627,-1.416190,-0.364831,1.259203


In [87]:
# Alvo de test (Ano 2021)
test_target = target_data.iloc[train_split:split_index + 1]
test_target

162    36.174
163    38.394
164    35.688
165    42.076
166    39.875
167    38.984
168    41.652
169    33.904
170    29.447
171    29.197
172    26.297
173    29.209
174    28.158
175    31.132
176    30.195
177    31.537
178    30.748
179    30.734
180    36.567
181    30.849
182    31.021
183    31.437
184    31.533
185    26.431
186    25.978
187    29.860
188    39.045
189    45.213
190    42.237
191    39.732
Name: Sergipe - Consumo de Cimento (t), dtype: float64

In [89]:
def validation_splitter(df, div_factor):
    split_factor = len(df)//div_factor
    positions_to_drop = []
    for i in range(split_factor):
        pos = df.shape[0] - (i*div_factor + 1)
        positions_to_drop.append(pos)
    
    return df.drop(positions_to_drop), df.iloc[positions_to_drop]

In [90]:
# Rede neural feed-forward com optmizador Estocástico
def neural_network_model(train_input, train_target, want_verbose=1, seed=0):
    if seed != 0:
        random.seed(seed)
        np.random.seed(seed)
        tf.random.set_seed(seed)
    # Aṕos 500 epochs sem grandes melhoras no val_loss, interrompe.
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', 
                                                      patience=500, 
                                                      verbose=want_verbose, 
                                                      restore_best_weights=True,
                                                      start_from_epoch=500)
    # Método estocástico e learning rate=0.005
    optimizer = tf.keras.optimizers.SGD(learning_rate=0.005)
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(2048, activation='tanh', input_shape=(train_input.shape[1],)),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(1024, activation='tanh'),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Dense(1024, activation='tanh'),
        tf.keras.layers.Dense(128, activation='tanh'),
        tf.keras.layers.Dense(1)
    ])
    train, train_val = validation_splitter(train_input, 6)
    target,target_val = validation_splitter(train_target, 6)
#     display(train)
#     display(train_val)
#     display(target)
#     display(target_val)
    model.compile(optimizer=optimizer, loss='mean_squared_error')   
    history = model.fit(train, 
                        target, 
                        epochs=10000,
                        validation_data=(train_val,target_val),
#                         validation_split=0.07,
                        callbacks=[early_stopping], 
                        verbose=want_verbose)
    return model, history

In [91]:
def get_a_good_seed(train_input, train_target, test_target, test_input):

    random_seeds = [random.randint(0, 2**32 - 1) for _ in range(25)]
    print(random_seeds)

    best_loss = float('inf')
    winner_seed = None
    i = 0
    for seed in random_seeds:
        print(f"\n\nStep: {i} ___________________________________________")
        i += 1

        model, history = neural_network_model(train_input, train_target, want_verbose=0, seed=seed)
        current_loss = min(history.history['val_loss'][500:])
        print(f"val_loss: {current_loss}")

        if current_loss < best_loss:
            best_loss = current_loss
            winner_seed = seed
            print(f"winner_seed: {winner_seed}")
            if winner_seed == 0.0:
                return winner_seed

    return winner_seed

In [92]:
winner_seed = get_a_good_seed(train_input, train_target, test_target, test_input)
print(f"\n\nfinal_seed: {winner_seed}")

[4275842662, 3763000187, 4258013487, 1108331720, 3957147699, 1265918454, 2465243761, 3192349287, 2088304250, 2492047360, 2584724075, 886796811, 2409995501, 653697939, 2104529171, 443762208, 3905717085, 2098735409, 1141887323, 53913117, 3465569774, 833137083, 2494261180, 1835415702, 484220725]


Step: 0 ___________________________________________
val_loss: 8.520135879516602
winner_seed: 4275842662


Step: 1 ___________________________________________
val_loss: 11.903101921081543


Step: 2 ___________________________________________
val_loss: 9.789751052856445


Step: 3 ___________________________________________
val_loss: 11.427339553833008


Step: 4 ___________________________________________
val_loss: 10.681568145751953


Step: 5 ___________________________________________
val_loss: 9.971320152282715


Step: 6 ___________________________________________
val_loss: 8.66307258605957


Step: 7 ___________________________________________
val_loss: 11.673982620239258


Step: 8 _____________

In [93]:
trained_model, history = neural_network_model(train_input, 
                                              train_target, 
                                              want_verbose=1, 
                                              seed=winner_seed)

Epoch 1/10000
5/5 [==============================] - 0s 25ms/step - loss: 1658.5546 - val_loss: 1491.2922
Epoch 2/10000
5/5 [==============================] - 0s 5ms/step - loss: 1197.6826 - val_loss: 1012.9887
Epoch 3/10000
5/5 [==============================] - 0s 5ms/step - loss: 309.6773 - val_loss: 40.1048
Epoch 4/10000
5/5 [==============================] - 0s 5ms/step - loss: 24.8866 - val_loss: 28.6067
Epoch 5/10000
5/5 [==============================] - 0s 5ms/step - loss: 15.9359 - val_loss: 19.5852
Epoch 6/10000
5/5 [==============================] - 0s 5ms/step - loss: 27.9613 - val_loss: 2200.7244
Epoch 7/10000
5/5 [==============================] - 0s 5ms/step - loss: 1868.9606 - val_loss: 897.6010
Epoch 8/10000
5/5 [==============================] - 0s 5ms/step - loss: 595.4797 - val_loss: 969.3063
Epoch 9/10000
5/5 [==============================] - 0s 5ms/step - loss: 566.4390 - val_loss: 152.0195
Epoch 10/10000
5/5 [==============================] - 0s 5ms/step - loss

5/5 [==============================] - 0s 5ms/step - loss: 8.3834 - val_loss: 15.5867
Epoch 82/10000
5/5 [==============================] - 0s 5ms/step - loss: 7.8395 - val_loss: 14.1013
Epoch 83/10000
5/5 [==============================] - 0s 5ms/step - loss: 7.7305 - val_loss: 15.1996
Epoch 84/10000
5/5 [==============================] - 0s 4ms/step - loss: 7.2582 - val_loss: 13.5403
Epoch 85/10000
5/5 [==============================] - 0s 4ms/step - loss: 8.0489 - val_loss: 15.3377
Epoch 86/10000
5/5 [==============================] - 0s 5ms/step - loss: 9.0001 - val_loss: 15.1996
Epoch 87/10000
5/5 [==============================] - 0s 5ms/step - loss: 7.4475 - val_loss: 15.4958
Epoch 88/10000
5/5 [==============================] - 0s 5ms/step - loss: 8.4319 - val_loss: 15.8349
Epoch 89/10000
5/5 [==============================] - 0s 5ms/step - loss: 7.9181 - val_loss: 13.8498
Epoch 90/10000
5/5 [==============================] - 0s 5ms/step - loss: 8.0688 - val_loss: 13.9005
Epoch

5/5 [==============================] - 0s 5ms/step - loss: 7.3296 - val_loss: 22.1248
Epoch 162/10000
5/5 [==============================] - 0s 5ms/step - loss: 8.1774 - val_loss: 14.1492
Epoch 163/10000
5/5 [==============================] - 0s 5ms/step - loss: 7.0450 - val_loss: 15.3477
Epoch 164/10000
5/5 [==============================] - 0s 5ms/step - loss: 7.6388 - val_loss: 21.0966
Epoch 165/10000
5/5 [==============================] - 0s 5ms/step - loss: 8.5323 - val_loss: 23.9086
Epoch 166/10000
5/5 [==============================] - 0s 5ms/step - loss: 8.2883 - val_loss: 16.8561
Epoch 167/10000
5/5 [==============================] - 0s 5ms/step - loss: 6.9691 - val_loss: 14.2586
Epoch 168/10000
5/5 [==============================] - 0s 5ms/step - loss: 7.7618 - val_loss: 14.3101
Epoch 169/10000
5/5 [==============================] - 0s 5ms/step - loss: 7.1715 - val_loss: 15.0927
Epoch 170/10000
5/5 [==============================] - 0s 5ms/step - loss: 7.6436 - val_loss: 14.4

5/5 [==============================] - 0s 5ms/step - loss: 6.3214 - val_loss: 12.9523
Epoch 242/10000
5/5 [==============================] - 0s 5ms/step - loss: 6.7067 - val_loss: 19.2807
Epoch 243/10000
5/5 [==============================] - 0s 4ms/step - loss: 8.9293 - val_loss: 14.8345
Epoch 244/10000
5/5 [==============================] - 0s 5ms/step - loss: 7.7180 - val_loss: 17.3085
Epoch 245/10000
5/5 [==============================] - 0s 5ms/step - loss: 7.8446 - val_loss: 15.2767
Epoch 246/10000
5/5 [==============================] - 0s 5ms/step - loss: 7.6001 - val_loss: 14.3085
Epoch 247/10000
5/5 [==============================] - 0s 5ms/step - loss: 7.0467 - val_loss: 13.9365
Epoch 248/10000
5/5 [==============================] - 0s 5ms/step - loss: 7.1381 - val_loss: 14.2763
Epoch 249/10000
5/5 [==============================] - 0s 5ms/step - loss: 6.8619 - val_loss: 13.9076
Epoch 250/10000
5/5 [==============================] - 0s 4ms/step - loss: 7.5991 - val_loss: 13.0

5/5 [==============================] - 0s 4ms/step - loss: 5.9719 - val_loss: 13.1555
Epoch 322/10000
5/5 [==============================] - 0s 4ms/step - loss: 5.9131 - val_loss: 12.3837
Epoch 323/10000
5/5 [==============================] - 0s 5ms/step - loss: 6.5158 - val_loss: 16.0453
Epoch 324/10000
5/5 [==============================] - 0s 5ms/step - loss: 8.3580 - val_loss: 23.0444
Epoch 325/10000
5/5 [==============================] - 0s 4ms/step - loss: 7.8467 - val_loss: 13.4149
Epoch 326/10000
5/5 [==============================] - 0s 5ms/step - loss: 5.9903 - val_loss: 12.4806
Epoch 327/10000
5/5 [==============================] - 0s 5ms/step - loss: 5.7347 - val_loss: 11.2962
Epoch 328/10000
5/5 [==============================] - 0s 4ms/step - loss: 5.5765 - val_loss: 13.2860
Epoch 329/10000
5/5 [==============================] - 0s 4ms/step - loss: 6.1762 - val_loss: 13.3976
Epoch 330/10000
5/5 [==============================] - 0s 5ms/step - loss: 6.5143 - val_loss: 12.6

...

5/5 [==============================] - 0s 4ms/step - loss: 4.5993 - val_loss: 12.8282
Epoch 1042/10000
5/5 [==============================] - 0s 5ms/step - loss: 5.2655 - val_loss: 12.5333
Epoch 1043/10000
5/5 [==============================] - 0s 5ms/step - loss: 5.7575 - val_loss: 12.8858
Epoch 1044/10000
5/5 [==============================] - 0s 4ms/step - loss: 4.9290 - val_loss: 11.9576
Epoch 1045/10000
5/5 [==============================] - 0s 5ms/step - loss: 5.0346 - val_loss: 12.0314
Epoch 1046/10000
5/5 [==============================] - 0s 5ms/step - loss: 5.2452 - val_loss: 12.0882
Epoch 1047/10000
5/5 [==============================] - 0s 5ms/step - loss: 4.7948 - val_loss: 12.2771
Epoch 1048/10000
5/5 [==============================] - 0s 4ms/step - loss: 4.9211 - val_loss: 12.6309
Epoch 1049/10000
5/5 [==============================] - 0s 4ms/step - loss: 5.1133 - val_loss: 14.3673
Epoch 1050/10000
5/5 [==============================] - 0s 4ms/step - loss: 5.0046 - val_l

5/5 [==============================] - 0s 4ms/step - loss: 4.7600 - val_loss: 11.6857
Epoch 1121/10000
5/5 [==============================] - 0s 5ms/step - loss: 4.7528 - val_loss: 14.9333
Epoch 1122/10000
5/5 [==============================] - 0s 4ms/step - loss: 4.9988 - val_loss: 14.2870
Epoch 1123/10000
5/5 [==============================] - 0s 4ms/step - loss: 5.1268 - val_loss: 12.4424
Epoch 1124/10000
5/5 [==============================] - 0s 5ms/step - loss: 4.5000 - val_loss: 11.5302
Epoch 1125/10000
5/5 [==============================] - 0s 4ms/step - loss: 4.5047 - val_loss: 19.6334
Epoch 1126/10000
5/5 [==============================] - 0s 5ms/step - loss: 5.5658 - val_loss: 10.8275
Epoch 1127/10000
5/5 [==============================] - 0s 5ms/step - loss: 4.5991 - val_loss: 13.1892
Epoch 1128/10000
5/5 [==============================] - 0s 5ms/step - loss: 5.0615 - val_loss: 11.7687
Epoch 1129/10000
5/5 [==============================] - 0s 5ms/step - loss: 4.6213 - val_l

5/5 [==============================] - 0s 5ms/step - loss: 4.8117 - val_loss: 11.7882
Epoch 1200/10000
5/5 [==============================] - 0s 5ms/step - loss: 4.9321 - val_loss: 13.7704
Epoch 1201/10000
5/5 [==============================] - 0s 4ms/step - loss: 4.6186 - val_loss: 11.7403
Epoch 1202/10000
5/5 [==============================] - 0s 4ms/step - loss: 4.7185 - val_loss: 16.4161
Epoch 1203/10000
5/5 [==============================] - 0s 4ms/step - loss: 5.3435 - val_loss: 13.0826
Epoch 1204/10000
5/5 [==============================] - 0s 4ms/step - loss: 4.7606 - val_loss: 12.1751
Epoch 1205/10000
5/5 [==============================] - 0s 4ms/step - loss: 4.1972 - val_loss: 12.0664
Epoch 1206/10000
5/5 [==============================] - 0s 4ms/step - loss: 4.9071 - val_loss: 14.0377
Epoch 1207/10000
5/5 [==============================] - 0s 5ms/step - loss: 4.9049 - val_loss: 14.4025
Epoch 1208/10000
5/5 [==============================] - 0s 5ms/step - loss: 4.5022 - val_l

5/5 [==============================] - 0s 5ms/step - loss: 4.0948 - val_loss: 11.9661
Epoch 1279/10000
5/5 [==============================] - 0s 5ms/step - loss: 4.4951 - val_loss: 11.5868
Epoch 1280/10000
5/5 [==============================] - 0s 5ms/step - loss: 4.1798 - val_loss: 11.6532
Epoch 1281/10000
5/5 [==============================] - 0s 5ms/step - loss: 4.6159 - val_loss: 11.5814
Epoch 1282/10000
5/5 [==============================] - 0s 5ms/step - loss: 4.5334 - val_loss: 15.1357
Epoch 1283/10000
5/5 [==============================] - 0s 5ms/step - loss: 5.7596 - val_loss: 16.4170
Epoch 1284/10000
5/5 [==============================] - 0s 4ms/step - loss: 5.7686 - val_loss: 14.2857
Epoch 1285/10000
5/5 [==============================] - 0s 5ms/step - loss: 4.6277 - val_loss: 15.5481
Epoch 1286/10000
5/5 [==============================] - 0s 5ms/step - loss: 4.7789 - val_loss: 15.2709
Epoch 1287/10000
5/5 [==============================] - 0s 5ms/step - loss: 4.7195 - val_l

5/5 [==============================] - 0s 5ms/step - loss: 5.1113 - val_loss: 19.0232
Epoch 1358/10000
5/5 [==============================] - 0s 5ms/step - loss: 6.5838 - val_loss: 15.3658
Epoch 1359/10000
5/5 [==============================] - 0s 5ms/step - loss: 5.5288 - val_loss: 12.2754
Epoch 1360/10000
5/5 [==============================] - 0s 5ms/step - loss: 4.9917 - val_loss: 11.7007
Epoch 1361/10000
5/5 [==============================] - 0s 5ms/step - loss: 5.0074 - val_loss: 11.3125
Epoch 1362/10000
5/5 [==============================] - 0s 5ms/step - loss: 4.4784 - val_loss: 12.7253
Epoch 1363/10000
5/5 [==============================] - 0s 5ms/step - loss: 4.6106 - val_loss: 11.4726
Epoch 1364/10000
5/5 [==============================] - 0s 4ms/step - loss: 4.5206 - val_loss: 11.4407
Epoch 1365/10000
5/5 [==============================] - 0s 5ms/step - loss: 4.7690 - val_loss: 12.5921
Epoch 1366/10000
5/5 [==============================] - 0s 5ms/step - loss: 4.5115 - val_l

In [94]:
def mae_mape_calculator(model, test_input, test_target, start_target):
    
    errors = []
    error_percent = []
    results_data = []

    for i in range(len(test_target)):
        prediction = model.predict(test_input.iloc[i:i+1])
        target = test_target[start_target + i]
        error = np.abs(prediction - target)
        errors.append(error)
        error_percent.append(error/target)
        results_data.append([f"Month-{i + 1}", 
                             prediction[0][0], 
                             target, 
                             error[0][0]])

    df_results = pd.DataFrame(results_data, columns=["Month", "Prediction", "Target", "Error"])

    mae = np.mean(errors)
    mape = np.mean(error_percent) 

    return df_results, mae, mape

In [95]:
df_results, mae, mape = mae_mape_calculator(trained_model, 
                                            test_input, 
                                            test_target, 
                                            train_split)
pd.set_option('display.max_columns', None)
df_results.T

1/1 [==============================] - 0s 13ms/step


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29
Month,Month-1,Month-2,Month-3,Month-4,Month-5,Month-6,Month-7,Month-8,Month-9,Month-10,Month-11,Month-12,Month-13,Month-14,Month-15,Month-16,Month-17,Month-18,Month-19,Month-20,Month-21,Month-22,Month-23,Month-24,Month-25,Month-26,Month-27,Month-28,Month-29,Month-30
Prediction,38.754044,38.69566,38.367805,38.335838,38.230377,37.818172,21.458702,19.756649,19.86887,12.050808,-2.987385,24.389454,-4.126945,-4.188384,-3.913354,-1.659755,4.074056,-4.194274,-4.202342,-4.195573,-4.206163,-4.23601,-4.359323,-4.510295,-4.923594,-5.230797,-5.859454,-7.206473,-8.245431,-9.894261
Target,36.174,38.394,35.688,42.076,39.875,38.984,41.652,33.904,29.447,29.197,26.297,29.209,28.158,31.132,30.195,31.537,30.748,30.734,36.567,30.849,31.021,31.437,31.533,26.431,25.978,29.86,39.045,45.213,42.237,39.732
Error,2.580044,0.301659,2.679806,3.740162,1.644623,1.165829,20.193298,14.14735,9.578131,17.146193,29.284386,4.819546,32.284946,35.320385,34.108353,33.196754,26.673943,34.928272,40.769344,35.044575,35.227165,35.673012,35.892323,30.941296,30.901596,35.090797,44.904453,52.419476,50.48243,49.626259


In [96]:
display(mae)
display(mape)

26.02555

0.79411304

In [97]:
def year_mae_mape_calculator(model, test_input, test_target, start_target):
    
    errors = []
    error_percent = []
    
    target_sum = 0
    prediction_sum = 0
    
    for i in range(len(test_target)):
        if i % 12 == 0 and i != 0:
            error = np.abs(target_sum - prediction_sum)
            errors.append(error)
            error_percent.append(error / target_sum)
            print(f"Ano-{i%12}: |Prediction{prediction_sum} - Target[{target_sum}]| =  Error: {error}; MAPE:{abs(prediction_sum - target_sum)/target_sum}")
            target_sum = 0
            prediction_sum = 0
            
        prediction = model.predict(test_input.iloc[i:i+1])
        target_sum += test_target[start_target + i]
        prediction_sum += prediction
        
    error = np.abs(target_sum - prediction_sum)
    errors.append(error)
    error_percent.append(error / target_sum)
    print(f"Ano-{i%12}: |Prediction{prediction_sum} - Target[{target_sum}]| =  Error: {error}; MAPE:{abs(prediction_sum - target_sum)/target_sum}")
        
    mae = np.mean(errors)
    mape = np.mean(error_percent) 

    return errors, mae, mape

In [98]:
errors, mae, mape = year_mae_mape_calculator(trained_model, 
                                             test_input, 
                                             test_target, 
                                             train_split)
display(errors)
display(mae)
display(mape)

1/1 [==============================] - 0s 13ms/step
Ano-0: |Prediction[[324.739]] - Target[420.89700000000005]| =  Error: [[96.15799]]; MAPE:[[0.22845967]]
1/1 [==============================] - 0s 13ms/step
Ano-0: |Prediction[[-39.71836]] - Target[370.34200000000004]| =  Error: [[410.06036]]; MAPE:[[1.1072477]]
1/1 [==============================] - 0s 14ms/step
Ano-5: |Prediction[[-41.36001]] - Target[222.065]| =  Error: [[263.42502]]; MAPE:[[1.1862519]]


[array([[96.15799]], dtype=float32),
 array([[410.06036]], dtype=float32),
 array([[263.42502]], dtype=float32)]

256.5478

0.8406531